In [7]:
from llama_index.core import Document
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import TextNode
from typing import List, Dict, Any
import re

class LlamaIndexSplitter:
    def __init__(
        self, 
        chunk_size: int = 1024, 
        chunk_overlap: int = 20,
        **kwargs: Any
    ) -> None:
        """Initialize a LlamaIndexSplitter."""
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.parser = SimpleNodeParser.from_defaults(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
        )

    def split_document(self, document: Document) -> List[TextNode]:
        """Split a document into nodes."""
        nodes = self.parser.get_nodes_from_documents([document])
        
        # Process each node to extract additional metadata
        for node in nodes:
            self._process_node(node)
        
        return nodes

    def _process_node(self, node: TextNode) -> None:
        """Process a node to extract additional metadata."""
        # Extract main topic (usually the first header)
        main_topic = self._extract_main_topic(node.text)
        
        # Extract code blocks
        code_blocks = self._extract_code_blocks(node.text)
        
        # Update node metadata
        node.metadata.update({
            "main_topic": main_topic,
            "code_blocks": code_blocks,
            "has_code": len(code_blocks) > 0
        })

    def _extract_main_topic(self, text: str) -> str:
        """Extract the main topic from the text."""
        lines = text.split('\n')
        for line in lines:
            if line.startswith('#'):
                return line.strip('# ').strip()
        return ""

    def _extract_code_blocks(self, text: str) -> List[str]:
        """Extract code blocks from the text."""
        code_block_pattern = r'```[\s\S]*?```'
        return re.findall(code_block_pattern, text)

    def split_text(self, text: str) -> List[Dict[str, Any]]:
        """Split text into chunks with metadata."""
        doc = Document(text=text)
        nodes = self.split_document(doc)
        
        return [
            {
                "content": node.text,
                "metadata": node.metadata
            }
            for node in nodes
        ]

# Example usage
if __name__ == "__main__":
    splitter = LlamaIndexSplitter(chunk_size=1000, chunk_overlap=200)
    
    sample_text = """
    # LlamaIndex Tutorial
    
    LlamaIndex is a powerful tool for building AI applications.
    
    ## Installation
    
    You can install LlamaIndex using pip:
    
    ```
    pip install llama-index
    ```
    
    ## Basic Usage
    
    Here's a simple example:
    
    ```python
    from llama_index import SimpleDirectoryReader, GPTListIndex, GPTVectorStoreIndex, LLMPredictor, PromptHelper
    from langchain.chat_models import ChatOpenAI
    
    # Load documents
    documents = SimpleDirectoryReader('data').load_data()
    
    # Create index
    index = GPTVectorStoreIndex.from_documents(documents)
    
    # Query index
    response = index.query("What is LlamaIndex?")
    print(response)
    ```
    """
    
    chunks = splitter.split_text(sample_text)


In [10]:
from llama_index.core import Document
from typing import List, Dict, Any
import re

class LlamaIndexSplitter:
    def __init__(self):
        """Initialize a LlamaIndexSplitter."""
        self.header_patterns = [
            (r'^# (.+)', 'head1'),
            (r'^## (.+)', 'head2'),
            (r'^### (.+)', 'head3'),
            (r'^#### (.+)', 'head4'),
        ]
        self.article_pattern = r'^第[一二三四五六七八九十百千]+条'

    def split_document(self, text: str) -> List[Document]:
        """Split a document into smaller documents based on headers and articles."""
        lines = text.split('\n')
        documents = []
        current_headers: Dict[str, str] = {}
        current_content = []

        for line in lines:
            header_match = self._match_header(line)
            if header_match:
                level, content = header_match
                current_headers[level] = content.strip()
                if current_content:
                    documents.append(self._create_document(current_content, current_headers))
                    current_content = []
            elif re.match(self.article_pattern, line):
                if current_content:
                    documents.append(self._create_document(current_content, current_headers))
                current_content = [line]
            else:
                current_content.append(line)

        if current_content:
            documents.append(self._create_document(current_content, current_headers))

        return documents

    def _match_header(self, line: str) -> Any:
        """Match a line against header patterns."""
        for pattern, level in self.header_patterns:
            match = re.match(pattern, line)
            if match:
                return level, match.group(1)
        return None

    def _create_document(self, content: List[str], headers: Dict[str, str]) -> Document:
        """Create a Document from content and headers."""
        return Document(
            text='\n'.join(content).strip(),
            metadata=headers.copy()
        )

# Example usage
if __name__ == "__main__":
    splitter = LlamaIndexSplitter()
    
    sample_text = """
# 中华人民共和国宪法

1982年12月4日 第五届全国人民代表大会第五次会议通过

1982年12月4日 全国人民代表大会公告公布施行

1988年4月12日 第七届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

1993年3月29日 第八届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

1999年3月15日 第九届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》

2004年3月14日 第十届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》

2018年3月11日 第十三届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

<!-- INFO END -->

## 序言

中国是世界上历史最悠久的国家之一。中国各族人民共同创造了光辉灿烂的文化，具有光荣的革命传统。

一八四○年以后，封建的中国逐渐变成半殖民地、半封建的国家。中国人民为国家独立、民族解放和民主自由进行了前仆后继的英勇奋斗。

二十世纪，中国发生了翻天覆地的伟大历史变革。

一九一一年孙中山先生领导的辛亥革命，废除了封建帝制，创立了中华民国。但是，中国人民反对帝国主义和封建主义的历史任务还没有完成。

一九四九年，以毛泽东主席为领袖的中国共产党领导中国各族人民，在经历了长期的艰难曲折的武装斗争和其他形式的斗争以后，终于推翻了帝国主义、封建主义和官僚资本主义的统治，取得了新民主主义革命的伟大胜利，建立了中华人民共和国。从此，中国人民掌握了国家的权力，成为国家的主人。

<!-- FORCE BREAK -->

中华人民共和国成立以后，我国社会逐步实现了由新民主主义到社会主义的过渡。生产资料私有制的社会主义改造已经完成，人剥削人的制度已经消灭，社会主义制度已经确立。工人阶级领导的、以工农联盟为基础的人民民主专政，实质上即无产阶级专政，得到巩固和发展。中国人民和中国人民解放军战胜了帝国主义、霸权主义的侵略、破坏和武装挑衅，维护了国家的独立和安全，增强了国防。经济建设取得了重大的成就，独立的、比较完整的社会主义工业体系已经基本形成，农业生产显著提高。教育、科学、文化等事业有了很大的发展，社会主义思想教育取得了明显的成效。广大人民的生活有了较大的改善。

<!-- FORCE BREAK -->

中国新民主主义革命的胜利和社会主义事业的成就，是中国共产党领导中国各族人民，在马克思列宁主义、毛泽东思想的指引下，坚持真理，修正错误，战胜许多艰难险阻而取得的。我国将长期处于社会主义初级阶段。国家的根本任务是，沿着中国特色社会主义道路，集中力量进行社会主义现代化建设。中国各族人民将继续在中国共产党领导下，在马克思列宁主义、毛泽东思想、邓小平理论、“三个代表”重要思想、科学发展观、习近平新时代中国特色社会主义思想指引下，坚持人民民主专政，坚持社会主义道路，坚持改革开放，不断完善社会主义的各项制度，发展社会主义市场经济，发展社会主义民主，健全社会主义法治，贯彻新发展理念，自力更生，艰苦奋斗，逐步实现工业、农业、国防和科学技术的现代化，推动物质文明、政治文明、精神文明、社会文明、生态文明协调发展，把我国建设成为富强民主文明和谐美丽的社会主义现代化强国，实现中华民族伟大复兴。

<!-- FORCE BREAK -->

在我国，剥削阶级作为阶级已经消灭，但是阶级斗争还将在一定范围内长期存在。中国人民对敌视和破坏我国社会主义制度的国内外的敌对势力和敌对分子，必须进行斗争。

台湾是中华人民共和国的神圣领土的一部分。完成统一祖国的大业是包括台湾同胞在内的全中国人民的神圣职责。

社会主义的建设事业必须依靠工人、农民和知识分子，团结一切可以团结的力量。在长期的革命、建设、改革过程中，已经结成由中国共产党领导的，有各民主党派和各人民团体参加的，包括全体社会主义劳动者、社会主义事业的建设者、拥护社会主义的爱国者、拥护祖国统一和致力于中华民族伟大复兴的爱国者的广泛的爱国统一战线，这个统一战线将继续巩固和发展。中国人民政治协商会议是有广泛代表性的统一战线组织，过去发挥了重要的历史作用，今后在国家政治生活、社会生活和对外友好活动中，在进行社会主义现代化建设、维护国家的统一和团结的斗争中，将进一步发挥它的重要作用。中国共产党领导的多党合作和政治协商制度将长期存在和发展。

<!-- FORCE BREAK -->

中华人民共和国是全国各族人民共同缔造的统一的多民族国家。平等团结互助和谐的社会主义民族关系已经确立，并将继续加强。在维护民族团结的斗争中，要反对大民族主义，主要是大汉族主义，也要反对地方民族主义。国家尽一切努力，促进全国各民族的共同繁荣。

中国革命、建设、改革的成就是同世界人民的支持分不开的。中国的前途是同世界的前途紧密地联系在一起的。中国坚持独立自主的对外政策，坚持互相尊重主权和领土完整、互不侵犯、互不干涉内政、平等互利、和平共处的五项原则，坚持和平发展道路，坚持互利共赢开放战略，发展同各国的外交关系和经济、文化交流，推动构建人类命运共同体；坚持反对帝国主义、霸权主义、殖民主义，加强同世界各国人民的团结，支持被压迫民族和发展中国家争取和维护民族独立、发展民族经济的正义斗争，为维护世界和平和促进人类进步事业而努力。

本宪法以法律的形式确认了中国各族人民奋斗的成果，规定了国家的根本制度和根本任务，是国家的根本法，具有最高的法律效力。全国各族人民、一切国家机关和武装力量、各政党和各社会团体、各企业事业组织，都必须以宪法为根本的活动准则，并且负有维护宪法尊严、保证宪法实施的职责。

## 第一章 总纲

第一条 中华人民共和国是工人阶级领导的、以工农联盟为基础的人民民主专政的社会主义国家。

社会主义制度是中华人民共和国的根本制度。中国共产党领导是中国特色社会主义最本质的特征。禁止任何组织或者个人破坏社会主义制度。

第二条 中华人民共和国的一切权力属于人民。

人民行使国家权力的机关是全国人民代表大会和地方各级人民代表大会。

人民依照法律规定，通过各种途径和形式，管理国家事务，管理经济和文化事业，管理社会事务。

第三条 中华人民共和国的国家机构实行民主集中制的原则。

全国人民代表大会和地方各级人民代表大会都由民主选举产生，对人民负责，受人民监督。

国家行政机关、监察机关、审判机关、检察机关都由人民代表大会产生，对它负责，受它监督。

中央和地方的国家机构职权的划分，遵循在中央的统一领导下，充分发挥地方的主动性、积极性的原则。

第四条 中华人民共和国各民族一律平等。国家保障各少数民族的合法的权利和利益，维护和发展各民族的平等团结互助和谐关系。禁止对任何民族的歧视和压迫，禁止破坏民族团结和制造民族分裂的行为。

国家根据各少数民族的特点和需要，帮助各少数民族地区加速经济和文化的发展。

各少数民族聚居的地方实行区域自治，设立自治机关，行使自治权。各民族自治地方都是中华人民共和国不可分离的部分。

各民族都有使用和发展自己的语言文字的自由，都有保持或者改革自己的风俗习惯的自由。

第五条 中华人民共和国实行依法治国，建设社会主义法治国家。

国家维护社会主义法制的统一和尊严。

一切法律、行政法规和地方性法规都不得同宪法相抵触。

一切国家机关和武装力量、各政党和各社会团体、各企业事业组织都必须遵守宪法和法律。一切违反宪法和法律的行为，必须予以追究。

任何组织或者个人都不得有超越宪法和法律的特权。

第六条 中华人民共和国的社会主义经济制度的基础是生产资料的社会主义公有制，即全民所有制和劳动群众集体所有制。社会主义公有制消灭人剥削人的制度，实行各尽所能、按劳分配的原则。

国家在社会主义初级阶段，坚持公有制为主体、多种所有制经济共同发展的基本经济制度，坚持按劳分配为主体、多种分配方式并存的分配制度。

第七条 国有经济，即社会主义全民所有制经济，是国民经济中的主导力量。国家保障国有经济的巩固和发展。

第八条 农村集体经济组织实行家庭承包经营为基础、统分结合的双层经营体制。农村中的生产、供销、信用、消费等各种形式的合作经济，是社会主义劳动群众集体所有制经济。参加农村集体经济组织的劳动者，有权在法律规定的范围内经营自留地、自留山、家庭副业和饲养自留畜。

城镇中的手工业、工业、建筑业、运输业、商业、服务业等行业的各种形式的合作经济，都是社会主义劳动群众集体所有制经济。

国家保护城乡集体经济组织的合法的权利和利益，鼓励、指导和帮助集体经济的发展。

第九条 矿藏、水流、森林、山岭、草原、荒地、滩涂等自然资源，都属于国家所有，即全民所有；由法律规定属于集体所有的森林和山岭、草原、荒地、滩涂除外。

国家保障自然资源的合理利用，保护珍贵的动物和植物。禁止任何组织或者个人用任何手段侵占或者破坏自然资源。

第十条 城市的土地属于国家所有。

农村和城市郊区的土地，除由法律规定属于国家所有的以外，属于集体所有；宅基地和自留地、自留山，也属于集体所有。

国家为了公共利益的需要，可以依照法律规定对土地实行征收或者征用并给予补偿。

任何组织或者个人不得侵占、买卖或者以其他形式非法转让土地。土地的使用权可以依照法律的规定转让。

一切使用土地的组织和个人必须合理地利用土地。

第十一条 在法律规定范围内的个体经济、私营经济等非公有制经济，是社会主义市场经济的重要组成部分。

国家保护个体经济、私营经济等非公有制经济的合法的权利和利益。国家鼓励、支持和引导非公有制经济的发展，并对非公有制经济依法实行监督和管理。

第十二条 社会主义的公共财产神圣不可侵犯。

国家保护社会主义的公共财产。禁止任何组织或者个人用任何手段侵占或者破坏国家的和集体的财产。

第十三条 公民的合法的私有财产不受侵犯。

国家依照法律规定保护公民的私有财产权和继承权。

国家为了公共利益的需要，可以依照法律规定对公民的私有财产实行征收或者征用并给予补偿。

第十四条 国家通过提高劳动者的积极性和技术水平，推广先进的科学技术，完善经济管理体制和企业经营管理制度，实行各种形式的社会主义责任制，改进劳动组织，以不断提高劳动生产率和经济效益，发展社会生产力。

国家厉行节约，反对浪费。

国家合理安排积累和消费，兼顾国家、集体和个人的利益，在发展生产的基础上，逐步改善人民的物质生活和文化生活。

国家建立健全同经济发展水平相适应的社会保障制度。

第十五条 国家实行社会主义市场经济。

国家加强经济立法，完善宏观调控。

国家依法禁止任何组织或者个人扰乱社会经济秩序。

第十六条 国有企业在法律规定的范围内有权自主经营。

国有企业依照法律规定，通过职工代表大会和其他形式，实行民主管理。

第十七条 集体经济组织在遵守有关法律的前提下，有独立进行经济活动的自主权。

集体经济组织实行民主管理，依照法律规定选举和罢免管理人员，决定经营管理的重大问题。

第十八条 中华人民共和国允许外国的企业和其他经济组织或者个人依照中华人民共和国法律的规定在中国投资，同中国的企业或者其他经济组织进行各种形式的经济合作。

在中国境内的外国企业和其他外国经济组织以及中外合资经营的企业，都必须遵守中华人民共和国的法律。它们的合法的权利和利益受中华人民共和国法律的保护。

第十九条 国家发展社会主义的教育事业，提高全国人民的科学文化水平。

国家举办各种学校，普及初等义务教育，发展中等教育、职业教育和高等教育，并且发展学前教育。

国家发展各种教育设施，扫除文盲，对工人、农民、国家工作人员和其他劳动者进行政治、文化、科学、技术、业务的教育，鼓励自学成才。

国家鼓励集体经济组织、国家企业事业组织和其他社会力量依照法律规定举办各种教育事业。

国家推广全国通用的普通话。

第二十条 国家发展自然科学和社会科学事业，普及科学和技术知识，奖励科学研究成果和技术发明创造。

第二十一条 国家发展医疗卫生事业，发展现代医药和我国传统医药，鼓励和支持农村集体经济组织、国家企业事业组织和街道组织举办各种医疗卫生设施，开展群众性的卫生活动，保护人民健康。

国家发展体育事业，开展群众性的体育活动，增强人民体质。

第二十二条 国家发展为人民服务、为社会主义服务的文学艺术事业、新闻广播电视事业、出版发行事业、图书馆博物馆文化馆和其他文化事业，开展群众性的文化活动。

国家保护名胜古迹、珍贵文物和其他重要历史文化遗产。

第二十三条 国家培养为社会主义服务的各种专业人才，扩大知识分子的队伍，创造条件，充分发挥他们在社会主义现代化建设中的作用。

第二十四条 国家通过普及理想教育、道德教育、文化教育、纪律和法制教育，通过在城乡不同范围的群众中制定和执行各种守则、公约，加强社会主义精神文明的建设。

国家倡导社会主义核心价值观，提倡爱祖国、爱人民、爱劳动、爱科学、爱社会主义的公德，在人民中进行爱国主义、集体主义和国际主义、共产主义的教育，进行辩证唯物主义和历史唯物主义的教育，反对资本主义的、封建主义的和其他的腐朽思想。

第二十五条 国家推行计划生育，使人口的增长同经济和社会发展计划相适应。

第二十六条 国家保护和改善生活环境和生态环境，防治污染和其他公害。

国家组织和鼓励植树造林，保护林木。

第二十七条 一切国家机关实行精简的原则，实行工作责任制，实行工作人员的培训和考核制度，不断提高工作质量和工作效率，反对官僚主义。

一切国家机关和国家工作人员必须依靠人民的支持，经常保持同人民的密切联系，倾听人民的意见和建议，接受人民的监督，努力为人民服务。

国家工作人员就职时应当依照法律规定公开进行宪法宣誓。

第二十八条 国家维护社会秩序，镇压叛国和其他危害国家安全的犯罪活动，制裁危害社会治安、破坏社会主义经济和其他犯罪的活动，惩办和改造犯罪分子。

第二十九条 中华人民共和国的武装力量属于人民。它的任务是巩固国防，抵抗侵略，保卫祖国，保卫人民的和平劳动，参加国家建设事业，努力为人民服务。

国家加强武装力量的革命化、现代化、正规化的建设，增强国防力量。

第三十条 中华人民共和国的行政区域划分如下：

（一）全国分为省、自治区、直辖市；

（二）省、自治区分为自治州、县、自治县、市；

（三）县、自治县分为乡、民族乡、镇。

直辖市和较大的市分为区、县。自治州分为县、自治县、市。

自治区、自治州、自治县都是民族自治地方。

第三十一条 国家在必要时得设立特别行政区。在特别行政区内实行的制度按照具体情况由全国人民代表大会以法律规定。

第三十二条 中华人民共和国保护在中国境内的外国人的合法权利和利益，在中国境内的外国人必须遵守中华人民共和国的法律。

中华人民共和国对于因为政治原因要求避难的外国人，可以给予受庇护的权利。

## 第二章 公民的基本权利和义务

第三十三条 凡具有中华人民共和国国籍的人都是中华人民共和国公民。

中华人民共和国公民在法律面前一律平等。

国家尊重和保障人权。

任何公民享有宪法和法律规定的权利，同时必须履行宪法和法律规定的义务。

第三十四条 中华人民共和国年满十八周岁的公民，不分民族、种族、性别、职业、家庭出身、宗教信仰、教育程度、财产状况、居住期限，都有选举权和被选举权；但是依照法律被剥夺政治权利的人除外。

第三十五条 中华人民共和国公民有言论、出版、集会、结社、游行、示威的自由。

第三十六条 中华人民共和国公民有宗教信仰自由。

任何国家机关、社会团体和个人不得强制公民信仰宗教或者不信仰宗教，不得歧视信仰宗教的公民和不信仰宗教的公民。

国家保护正常的宗教活动。任何人不得利用宗教进行破坏社会秩序、损害公民身体健康、妨碍国家教育制度的活动。

宗教团体和宗教事务不受外国势力的支配。

第三十七条 中华人民共和国公民的人身自由不受侵犯。

任何公民，非经人民检察院批准或者决定或者人民法院决定，并由公安机关执行，不受逮捕。

禁止非法拘禁和以其他方法非法剥夺或者限制公民的人身自由，禁止非法搜查公民的身体。

第三十八条 中华人民共和国公民的人格尊严不受侵犯。禁止用任何方法对公民进行侮辱、诽谤和诬告陷害。

第三十九条 中华人民共和国公民的住宅不受侵犯。禁止非法搜查或者非法侵入公民的住宅。

第四十条 中华人民共和国公民的通信自由和通信秘密受法律的保护。除因国家安全或者追查刑事犯罪的需要，由公安机关或者检察机关依照法律规定的程序对通信进行检查外，任何组织或者个人不得以任何理由侵犯公民的通信自由和通信秘密。

第四十一条 中华人民共和国公民对于任何国家机关和国家工作人员，有提出批评和建议的权利；对于任何国家机关和国家工作人员的违法失职行为，有向有关国家机关提出申诉、控告或者检举的权利，但是不得捏造或者歪曲事实进行诬告陷害。

对于公民的申诉、控告或者检举，有关国家机关必须查清事实，负责处理。任何人不得压制和打击报复。

由于国家机关和国家工作人员侵犯公民权利而受到损失的人，有依照法律规定取得赔偿的权利。

第四十二条 中华人民共和国公民有劳动的权利和义务。

国家通过各种途径，创造劳动就业条件，加强劳动保护，改善劳动条件，并在发展生产的基础上，提高劳动报酬和福利待遇。

劳动是一切有劳动能力的公民的光荣职责。国有企业和城乡集体经济组织的劳动者都应当以国家主人翁的态度对待自己的劳动。国家提倡社会主义劳动竞赛，奖励劳动模范和先进工作者。国家提倡公民从事义务劳动。

国家对就业前的公民进行必要的劳动就业训练。

第四十三条 中华人民共和国劳动者有休息的权利。

国家发展劳动者休息和休养的设施，规定职工的工作时间和休假制度。

第四十四条 国家依照法律规定实行企业事业组织的职工和国家机关工作人员的退休制度。退休人员的生活受到国家和社会的保障。

第四十五条 中华人民共和国公民在年老、疾病或者丧失劳动能力的情况下，有从国家和社会获得物质帮助的权利。国家发展为公民享受这些权利所需要的社会保险、社会救济和医疗卫生事业。

国家和社会保障残废军人的生活，抚恤烈士家属，优待军人家属。

国家和社会帮助安排盲、聋、哑和其他有残疾的公民的劳动、生活和教育。

第四十六条 中华人民共和国公民有受教育的权利和义务。

国家培养青年、少年、儿童在品德、智力、体质等方面全面发展。

第四十七条 中华人民共和国公民有进行科学研究、文学艺术创作和其他文化活动的自由。国家对于从事教育、科学、技术、文学、艺术和其他文化事业的公民的有益于人民的创造性工作，给以鼓励和帮助。

第四十八条 中华人民共和国妇女在政治的、经济的、文化的、社会的和家庭的生活等各方面享有同男子平等的权利。

国家保护妇女的权利和利益，实行男女同工同酬，培养和选拔妇女干部。

第四十九条 婚姻、家庭、母亲和儿童受国家的保护。

夫妻双方有实行计划生育的义务。

父母有抚养教育未成年子女的义务，成年子女有赡养扶助父母的义务。

禁止破坏婚姻自由，禁止虐待老人、妇女和儿童。

第五十条 中华人民共和国保护华侨的正当的权利和利益，保护归侨和侨眷的合法的权利和利益。

第五十一条 中华人民共和国公民在行使自由和权利的时候，不得损害国家的、社会的、集体的利益和其他公民的合法的自由和权利。

第五十二条 中华人民共和国公民有维护国家统一和全国各民族团结的义务。

第五十三条 中华人民共和国公民必须遵守宪法和法律，保守国家秘密，爱护公共财产，遵守劳动纪律，遵守公共秩序，尊重社会公德。

第五十四条 中华人民共和国公民有维护祖国的安全、荣誉和利益的义务，不得有危害祖国的安全、荣誉和利益的行为。

第五十五条 保卫祖国、抵抗侵略是中华人民共和国每一个公民的神圣职责。

依照法律服兵役和参加民兵组织是中华人民共和国公民的光荣义务。

第五十六条 中华人民共和国公民有依照法律纳税的义务。

## 第三章 国家机构

### 第一节 全国人民代表大会

第五十七条 中华人民共和国全国人民代表大会是最高国家权力机关。它的常设机关是全国人民代表大会常务委员会。

第五十八条 全国人民代表大会和全国人民代表大会常务委员会行使国家立法权。

第五十九条 全国人民代表大会由省、自治区、直辖市、特别行政区和军队选出的代表组成。各少数民族都应当有适当名额的代表。

全国人民代表大会代表的选举由全国人民代表大会常务委员会主持。

全国人民代表大会代表名额和代表产生办法由法律规定。

第六十条 全国人民代表大会每届任期五年。

全国人民代表大会任期届满的两个月以前，全国人民代表大会常务委员会必须完成下届全国人民代表大会代表的选举。如果遇到不能进行选举的非常情况，由全国人民代表大会常务委员会以全体组成人员的三分之二以上的多数通过，可以推迟选举，延长本届全国人民代表大会的任期。在非常情况结束后一年内，必须完成下届全国人民代表大会代表的选举。

第六十一条 全国人民代表大会会议每年举行一次，由全国人民代表大会常务委员会召集。如果全国人民代表大会常务委员会认为必要，或者有五分之一以上的全国人民代表大会代表提议，可以临时召集全国人民代表大会会议。

全国人民代表大会举行会议的时候，选举主席团主持会议。

第六十二条 全国人民代表大会行使下列职权：

（一）修改宪法；

（二）监督宪法的实施；

（三）制定和修改刑事、民事、国家机构的和其他的基本法律；

（四）选举中华人民共和国主席、副主席；

（五）根据中华人民共和国主席的提名，决定国务院总理的人选；根据国务院总理的提名，决定国务院副总理、国务委员、各部部长、各委员会主任、审计长、秘书长的人选；

（六）选举中央军事委员会主席；根据中央军事委员会主席的提名，决定中央军事委员会其他组成人员的人选；

（七）选举国家监察委员会主任；

（八）选举最高人民法院院长；

（九）选举最高人民检察院检察长；

（十）审查和批准国民经济和社会发展计划和计划执行情况的报告；

（十一）审查和批准国家的预算和预算执行情况的报告；

（十二）改变或者撤销全国人民代表大会常务委员会不适当的决定；

（十三）批准省、自治区和直辖市的建置；

（十四）决定特别行政区的设立及其制度；

（十五）决定战争和和平的问题；

（十六）应当由最高国家权力机关行使的其他职权。

第六十三条 全国人民代表大会有权罢免下列人员：

（一）中华人民共和国主席、副主席；

（二）国务院总理、副总理、国务委员、各部部长、各委员会主任、审计长、秘书长；

（三）中央军事委员会主席和中央军事委员会其他组成人员；

（四）国家监察委员会主任；

（五）最高人民法院院长；

（六）最高人民检察院检察长。

第六十四条 宪法的修改，由全国人民代表大会常务委员会或者五分之一以上的全国人民代表大会代表提议，并由全国人民代表大会以全体代表的三分之二以上的多数通过。

法律和其他议案由全国人民代表大会以全体代表的过半数通过。

第六十五条 全国人民代表大会常务委员会由下列人员组成：

委员长，

副委员长若干人，

秘书长，

委员若干人。

全国人民代表大会常务委员会组成人员中，应当有适当名额的少数民族代表。

全国人民代表大会选举并有权罢免全国人民代表大会常务委员会的组成人员。

全国人民代表大会常务委员会的组成人员不得担任国家行政机关、监察机关、审判机关和检察机关的职务。

第六十六条 全国人民代表大会常务委员会每届任期同全国人民代表大会每届任期相同，它行使职权到下届全国人民代表大会选出新的常务委员会为止。

委员长、副委员长连续任职不得超过两届。

第六十七条 全国人民代表大会常务委员会行使下列职权：

（一）解释宪法，监督宪法的实施；

（二）制定和修改除应当由全国人民代表大会制定的法律以外的其他法律；

（三）在全国人民代表大会闭会期间，对全国人民代表大会制定的法律进行部分补充和修改，但是不得同该法律的基本原则相抵触；

（四）解释法律；

（五）在全国人民代表大会闭会期间，审查和批准国民经济和社会发展计划、国家预算在执行过程中所必须作的部分调整方案；

（六）监督国务院、中央军事委员会、国家监察委员会、最高人民法院和最高人民检察院的工作；

（七）撤销国务院制定的同宪法、法律相抵触的行政法规、决定和命令；

（八）撤销省、自治区、直辖市国家权力机关制定的同宪法、法律和行政法规相抵触的地方性法规和决议；

（九）在全国人民代表大会闭会期间，根据国务院总理的提名，决定部长、委员会主任、审计长、秘书长的人选；

（十）在全国人民代表大会闭会期间，根据中央军事委员会主席的提名，决定中央军事委员会其他组成人员的人选；

（十一）根据国家监察委员会主任的提请，任免国家监察委员会副主任、委员；

（十二）根据最高人民法院院长的提请，任免最高人民法院副院长、审判员、审判委员会委员和军事法院院长；

（十三）根据最高人民检察院检察长的提请，任免最高人民检察院副检察长、检察员、检察委员会委员和军事检察院检察长，并且批准省、自治区、直辖市的人民检察院检察长的任免；

（十四）决定驻外全权代表的任免；

（十五）决定同外国缔结的条约和重要协定的批准和废除；

（十六）规定军人和外交人员的衔级制度和其他专门衔级制度；

（十七）规定和决定授予国家的勋章和荣誉称号；

（十八）决定特赦；

（十九）在全国人民代表大会闭会期间，如果遇到国家遭受武装侵犯或者必须履行国际间共同防止侵略的条约的情况，决定战争状态的宣布；

（二十）决定全国总动员或者局部动员；

（二十一）决定全国或者个别省、自治区、直辖市进入紧急状态；

（二十二）全国人民代表大会授予的其他职权。

第六十八条 全国人民代表大会常务委员会委员长主持全国人民代表大会常务委员会的工作，召集全国人民代表大会常务委员会会议。副委员长、秘书长协助委员长工作。

委员长、副委员长、秘书长组成委员长会议，处理全国人民代表大会常务委员会的重要日常工作。

第六十九条 全国人民代表大会常务委员会对全国人民代表大会负责并报告工作。

第七十条 全国人民代表大会设立民族委员会、宪法和法律委员会、财政经济委员会、教育科学文化卫生委员会、外事委员会、华侨委员会和其他需要设立的专门委员会。在全国人民代表大会闭会期间，各专门委员会受全国人民代表大会常务委员会的领导。

各专门委员会在全国人民代表大会和全国人民代表大会常务委员会领导下，研究、审议和拟订有关议案。

第七十一条 全国人民代表大会和全国人民代表大会常务委员会认为必要的时候，可以组织关于特定问题的调查委员会，并且根据调查委员会的报告，作出相应的决议。

调查委员会进行调查的时候，一切有关的国家机关、社会团体和公民都有义务向它提供必要的材料。

第七十二条 全国人民代表大会代表和全国人民代表大会常务委员会组成人员，有权依照法律规定的程序分别提出属于全国人民代表大会和全国人民代表大会常务委员会职权范围内的议案。

第七十三条 全国人民代表大会代表在全国人民代表大会开会期间，全国人民代表大会常务委员会组成人员在常务委员会开会期间，有权依照法律规定的程序提出对国务院或者国务院各部、各委员会的质询案。受质询的机关必须负责答复。

第七十四条 全国人民代表大会代表，非经全国人民代表大会会议主席团许可，在全国人民代表大会闭会期间非经全国人民代表大会常务委员会许可，不受逮捕或者刑事审判。

第七十五条 全国人民代表大会代表在全国人民代表大会各种会议上的发言和表决，不受法律追究。

第七十六条 全国人民代表大会代表必须模范地遵守宪法和法律，保守国家秘密，并且在自己参加的生产、工作和社会活动中，协助宪法和法律的实施。

全国人民代表大会代表应当同原选举单位和人民保持密切的联系，听取和反映人民的意见和要求，努力为人民服务。

第七十七条 全国人民代表大会代表受原选举单位的监督。原选举单位有权依照法律规定的程序罢免本单位选出的代表。

第七十八条 全国人民代表大会和全国人民代表大会常务委员会的组织和工作程序由法律规定。

### 第二节 中华人民共和国主席

第七十九条 中华人民共和国主席、副主席由全国人民代表大会选举。

有选举权和被选举权的年满四十五周岁的中华人民共和国公民可以被选为中华人民共和国主席、副主席。

中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。

第八十条 中华人民共和国主席根据全国人民代表大会的决定和全国人民代表大会常务委员会的决定，公布法律，任免国务院总理、副总理、国务委员、各部部长、各委员会主任、审计长、秘书长，授予国家的勋章和荣誉称号，发布特赦令，宣布进入紧急状态，宣布战争状态，发布动员令。

第八十一条 中华人民共和国主席代表中华人民共和国，进行国事活动，接受外国使节；根据全国人民代表大会常务委员会的决定，派遣和召回驻外全权代表，批准和废除同外国缔结的条约和重要协定。

第八十二条 中华人民共和国副主席协助主席工作。

中华人民共和国副主席受主席的委托，可以代行主席的部分职权。

第八十三条 中华人民共和国主席、副主席行使职权到下届全国人民代表大会选出的主席、副主席就职为止。

第八十四条 中华人民共和国主席缺位的时候，由副主席继任主席的职位。

中华人民共和国副主席缺位的时候，由全国人民代表大会补选。

中华人民共和国主席、副主席都缺位的时候，由全国人民代表大会补选；在补选以前，由全国人民代表大会常务委员会委员长暂时代理主席职位。

### 第三节 国务院

第八十五条 中华人民共和国国务院，即中央人民政府，是最高国家权力机关的执行机关，是最高国家行政机关。

第八十六条 国务院由下列人员组成：

总理，

副总理若干人，

国务委员若干人，

各部部长，

各委员会主任，

审计长，

秘书长。

国务院实行总理负责制。各部、各委员会实行部长、主任负责制。

国务院的组织由法律规定。

第八十七条 国务院每届任期同全国人民代表大会每届任期相同。

总理、副总理、国务委员连续任职不得超过两届。

第八十八条 总理领导国务院的工作。副总理、国务委员协助总理工作。

总理、副总理、国务委员、秘书长组成国务院常务会议。

总理召集和主持国务院常务会议和国务院全体会议。

第八十九条 国务院行使下列职权：

（一）根据宪法和法律，规定行政措施，制定行政法规，发布决定和命令；

（二）向全国人民代表大会或者全国人民代表大会常务委员会提出议案；

（三）规定各部和各委员会的任务和职责，统一领导各部和各委员会的工作，并且领导不属于各部和各委员会的全国性的行政工作；

（四）统一领导全国地方各级国家行政机关的工作，规定中央和省、自治区、直辖市的国家行政机关的职权的具体划分；

（五）编制和执行国民经济和社会发展计划和国家预算；

（六）领导和管理经济工作和城乡建设、生态文明建设；

（七）领导和管理教育、科学、文化、卫生、体育和计划生育工作；

（八）领导和管理民政、公安、司法行政等工作；

（九）管理对外事务，同外国缔结条约和协定；

（十）领导和管理国防建设事业；

（十一）领导和管理民族事务，保障少数民族的平等权利和民族自治地方的自治权利；

（十二）保护华侨的正当的权利和利益，保护归侨和侨眷的合法的权利和利益；

（十三）改变或者撤销各部、各委员会发布的不适当的命令、指示和规章；

（十四）改变或者撤销地方各级国家行政机关的不适当的决定和命令；

（十五）批准省、自治区、直辖市的区域划分，批准自治州、县、自治县、市的建置和区域划分；

（十六）依照法律规定决定省、自治区、直辖市的范围内部分地区进入紧急状态；

（十七）审定行政机构的编制，依照法律规定任免、培训、考核和奖惩行政人员；

（十八）全国人民代表大会和全国人民代表大会常务委员会授予的其他职权。

第九十条 国务院各部部长、各委员会主任负责本部门的工作；召集和主持部务会议或者委员会会议、委务会议，讨论决定本部门工作的重大问题。

各部、各委员会根据法律和国务院的行政法规、决定、命令，在本部门的权限内，发布命令、指示和规章。

第九十一条 国务院设立审计机关，对国务院各部门和地方各级政府的财政收支，对国家的财政金融机构和企业事业组织的财务收支，进行审计监督。

审计机关在国务院总理领导下，依照法律规定独立行使审计监督权，不受其他行政机关、社会团体和个人的干涉。

第九十二条 国务院对全国人民代表大会负责并报告工作；在全国人民代表大会闭会期间，对全国人民代表大会常务委员会负责并报告工作。

### 第四节 中央军事委员会

第九十三条 中华人民共和国中央军事委员会领导全国武装力量。

中央军事委员会由下列人员组成：

主席，

副主席若干人，

委员若干人。

中央军事委员会实行主席负责制。

中央军事委员会每届任期同全国人民代表大会每届任期相同。

第九十四条 中央军事委员会主席对全国人民代表大会和全国人民代表大会常务委员会负责。

### 第五节 地方各级人民代表大会和地方各级人民政府

第九十五条 省、直辖市、县、市、市辖区、乡、民族乡、镇设立人民代表大会和人民政府。

地方各级人民代表大会和地方各级人民政府的组织由法律规定。

自治区、自治州、自治县设立自治机关。自治机关的组织和工作根据宪法第三章第五节、第六节规定的基本原则由法律规定。

第九十六条 地方各级人民代表大会是地方国家权力机关。

县级以上的地方各级人民代表大会设立常务委员会。

第九十七条 省、直辖市、设区的市的人民代表大会代表由下一级的人民代表大会选举；县、不设区的市、市辖区、乡、民族乡、镇的人民代表大会代表由选民直接选举。

地方各级人民代表大会代表名额和代表产生办法由法律规定。

第九十八条 地方各级人民代表大会每届任期五年。

第九十九条 地方各级人民代表大会在本行政区域内，保证宪法、法律、行政法规的遵守和执行；依照法律规定的权限，通过和发布决议，审查和决定地方的经济建设、文化建设和公共事业建设的计划。

县级以上的地方各级人民代表大会审查和批准本行政区域内的国民经济和社会发展计划、预算以及它们的执行情况的报告；有权改变或者撤销本级人民代表大会常务委员会不适当的决定。

民族乡的人民代表大会可以依照法律规定的权限采取适合民族特点的具体措施。

第一百条 省、直辖市的人民代表大会和它们的常务委员会，在不同宪法、法律、行政法规相抵触的前提下，可以制定地方性法规，报全国人民代表大会常务委员会备案。

设区的市的人民代表大会和它们的常务委员会，在不同宪法、法律、行政法规和本省、自治区的地方性法规相抵触的前提下，可以依照法律规定制定地方性法规，报本省、自治区人民代表大会常务委员会批准后施行。

第一百零一条 地方各级人民代表大会分别选举并且有权罢免本级人民政府的省长和副省长、市长和副市长、县长和副县长、区长和副区长、乡长和副乡长、镇长和副镇长。

县级以上的地方各级人民代表大会选举并且有权罢免本级监察委员会主任、本级人民法院院长和本级人民检察院检察长。选出或者罢免人民检察院检察长，须报上级人民检察院检察长提请该级人民代表大会常务委员会批准。

第一百零二条 省、直辖市、设区的市的人民代表大会代表受原选举单位的监督；县、不设区的市、市辖区、乡、民族乡、镇的人民代表大会代表受选民的监督。

地方各级人民代表大会代表的选举单位和选民有权依照法律规定的程序罢免由他们选出的代表。

第一百零三条 县级以上的地方各级人民代表大会常务委员会由主任、副主任若干人和委员若干人组成，对本级人民代表大会负责并报告工作。

县级以上的地方各级人民代表大会选举并有权罢免本级人民代表大会常务委员会的组成人员。

县级以上的地方各级人民代表大会常务委员会的组成人员不得担任国家行政机关、监察机关、审判机关和检察机关的职务。

第一百零四条 县级以上的地方各级人民代表大会常务委员会讨论、决定本行政区域内各方面工作的重大事项；监督本级人民政府、监察委员会、人民法院和人民检察院的工作；撤销本级人民政府的不适当的决定和命令；撤销下一级人民代表大会的不适当的决议；依照法律规定的权限决定国家机关工作人员的任免；在本级人民代表大会闭会期间，罢免和补选上一级人民代表大会的个别代表。

第一百零五条 地方各级人民政府是地方各级国家权力机关的执行机关，是地方各级国家行政机关。

地方各级人民政府实行省长、市长、县长、区长、乡长、镇长负责制。

第一百零六条 地方各级人民政府每届任期同本级人民代表大会每届任期相同。

第一百零七条 县级以上地方各级人民政府依照法律规定的权限，管理本行政区域内的经济、教育、科学、文化、卫生、体育事业、城乡建设事业和财政、民政、公安、民族事务、司法行政、计划生育等行政工作，发布决定和命令，任免、培训、考核和奖惩行政工作人员。

乡、民族乡、镇的人民政府执行本级人民代表大会的决议和上级国家行政机关的决定和命令，管理本行政区域内的行政工作。

省、直辖市的人民政府决定乡、民族乡、镇的建置和区域划分。

第一百零八条 县级以上的地方各级人民政府领导所属各工作部门和下级人民政府的工作，有权改变或者撤销所属各工作部门和下级人民政府的不适当的决定。

第一百零九条 县级以上的地方各级人民政府设立审计机关。地方各级审计机关依照法律规定独立行使审计监督权，对本级人民政府和上一级审计机关负责。

第一百一十条 地方各级人民政府对本级人民代表大会负责并报告工作。县级以上的地方各级人民政府在本级人民代表大会闭会期间，对本级人民代表大会常务委员会负责并报告工作。

地方各级人民政府对上一级国家行政机关负责并报告工作。全国地方各级人民政府都是国务院统一领导下的国家行政机关，都服从国务院。

第一百一十一条 城市和农村按居民居住地区设立的居民委员会或者村民委员会是基层群众性自治组织。居民委员会、村民委员会的主任、副主任和委员由居民选举。居民委员会、村民委员会同基层政权的相互关系由法律规定。

居民委员会、村民委员会设人民调解、治安保卫、公共卫生等委员会，办理本居住地区的公共事务和公益事业，调解民间纠纷，协助维护社会治安，并且向人民政府反映群众的意见、要求和提出建议。

### 第六节 民族自治地方的自治机关

第一百一十二条 民族自治地方的自治机关是自治区、自治州、自治县的人民代表大会和人民政府。

第一百一十三条 自治区、自治州、自治县的人民代表大会中，除实行区域自治的民族的代表外，其他居住在本行政区域内的民族也应当有适当名额的代表。

自治区、自治州、自治县的人民代表大会常务委员会中应当有实行区域自治的民族的公民担任主任或者副主任。

第一百一十四条 自治区主席、自治州州长、自治县县长由实行区域自治的民族的公民担任。

第一百一十五条 自治区、自治州、自治县的自治机关行使宪法第三章第五节规定的地方国家机关的职权，同时依照宪法、民族区域自治法和其他法律规定的权限行使自治权，根据本地方实际情况贯彻执行国家的法律、政策。

第一百一十六条 民族自治地方的人民代表大会有权依照当地民族的政治、经济和文化的特点，制定自治条例和单行条例。自治区的自治条例和单行条例，报全国人民代表大会常务委员会批准后生效。自治州、自治县的自治条例和单行条例，报省或者自治区的人民代表大会常务委员会批准后生效，并报全国人民代表大会常务委员会备案。

第一百一十七条 民族自治地方的自治机关有管理地方财政的自治权。凡是依照国家财政体制属于民族自治地方的财政收入，都应当由民族自治地方的自治机关自主地安排使用。

第一百一十八条 民族自治地方的自治机关在国家计划的指导下，自主地安排和管理地方性的经济建设事业。

国家在民族自治地方开发资源、建设企业的时候，应当照顾民族自治地方的利益。

第一百一十九条 民族自治地方的自治机关自主地管理本地方的教育、科学、文化、卫生、体育事业，保护和整理民族的文化遗产，发展和繁荣民族文化。

第一百二十条 民族自治地方的自治机关依照国家的军事制度和当地的实际需要，经国务院批准，可以组织本地方维护社会治安的公安部队。

第一百二十一条 民族自治地方的自治机关在执行职务的时候，依照本民族自治地方自治条例的规定，使用当地通用的一种或者几种语言文字。

第一百二十二条 国家从财政、物资、技术等方面帮助各少数民族加速发展经济建设和文化建设事业。

国家帮助民族自治地方从当地民族中大量培养各级干部、各种专业人才和技术工人。

### 第七节 监察委员会

第一百二十三条 中华人民共和国各级监察委员会是国家的监察机关。

第一百二十四条 中华人民共和国设立国家监察委员会和地方各级监察委员会。

监察委员会由下列人员组成：

主任，

副主任若干人，

委员若干人。

监察委员会主任每届任期同本级人民代表大会每届任期相同。国家监察委员会主任连续任职不得超过两届。

监察委员会的组织和职权由法律规定。

第一百二十五条 中华人民共和国国家监察委员会是最高监察机关。

国家监察委员会领导地方各级监察委员会的工作，上级监察委员会领导下级监察委员会的工作。

第一百二十六条 国家监察委员会对全国人民代表大会和全国人民代表大会常务委员会负责。地方各级监察委员会对产生它的国家权力机关和上一级监察委员会负责。

第一百二十七条 监察委员会依照法律规定独立行使监察权，不受行政机关、社会团体和个人的干涉。

监察机关办理职务违法和职务犯罪案件，应当与审判机关、检察机关、执法部门互相配合，互相制约。

### 第八节 人民法院和人民检察院

第一百二十八条 中华人民共和国人民法院是国家的审判机关。

第一百二十九条 中华人民共和国设立最高人民法院、地方各级人民法院和军事法院等专门人民法院。

最高人民法院院长每届任期同全国人民代表大会每届任期相同，连续任职不得超过两届。

人民法院的组织由法律规定。

第一百三十条 人民法院审理案件，除法律规定的特别情况外，一律公开进行。被告人有权获得辩护。

第一百三十一条 人民法院依照法律规定独立行使审判权，不受行政机关、社会团体和个人的干涉。

第一百三十二条 最高人民法院是最高审判机关。

最高人民法院监督地方各级人民法院和专门人民法院的审判工作，上级人民法院监督下级人民法院的审判工作。

第一百三十三条 最高人民法院对全国人民代表大会和全国人民代表大会常务委员会负责。地方各级人民法院对产生它的国家权力机关负责。

第一百三十四条 中华人民共和国人民检察院是国家的法律监督机关。

第一百三十五条 中华人民共和国设立最高人民检察院、地方各级人民检察院和军事检察院等专门人民检察院。

最高人民检察院检察长每届任期同全国人民代表大会每届任期相同，连续任职不得超过两届。

人民检察院的组织由法律规定。

第一百三十六条 人民检察院依照法律规定独立行使检察权，不受行政机关、社会团体和个人的干涉。

第一百三十七条 最高人民检察院是最高检察机关。

最高人民检察院领导地方各级人民检察院和专门人民检察院的工作，上级人民检察院领导下级人民检察院的工作。

第一百三十八条 最高人民检察院对全国人民代表大会和全国人民代表大会常务委员会负责。地方各级人民检察院对产生它的国家权力机关和上级人民检察院负责。

第一百三十九条 各民族公民都有用本民族语言文字进行诉讼的权利。人民法院和人民检察院对于不通晓当地通用的语言文字的诉讼参与人，应当为他们翻译。

在少数民族聚居或者多民族共同居住的地区，应当用当地通用的语言进行审理；起诉书、判决书、布告和其他文书应当根据实际需要使用当地通用的一种或者几种文字。

第一百四十条 人民法院、人民检察院和公安机关办理刑事案件，应当分工负责，互相配合，互相制约，以保证准确有效地执行法律。

## 第四章 国旗、国歌、国徽、首都

第一百四十一条 中华人民共和国国旗是五星红旗。

中华人民共和国国歌是《义勇军进行曲》。

第一百四十二条 中华人民共和国国徽，中间是五星照耀下的天安门，周围是谷穗和齿轮。

第一百四十三条 中华人民共和国首都是北京。
    """
    
    documents = splitter.split_document(sample_text)
    
    for i, doc in enumerate(documents):
        print(f"Document {i+1}:")
        print(f"Content: {doc.text}")
        print(f"Metadata: {doc.metadata}")
        print()

Document 1:
Content: 
Metadata: {'head1': '中华人民共和国宪法'}

Document 2:
Content: 1982年12月4日 第五届全国人民代表大会第五次会议通过

1982年12月4日 全国人民代表大会公告公布施行

1988年4月12日 第七届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

1993年3月29日 第八届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

1999年3月15日 第九届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》

2004年3月14日 第十届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》

2018年3月11日 第十三届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》

<!-- INFO END -->
Metadata: {'head1': '中华人民共和国宪法', 'head2': '序言'}

Document 3:
Content: 中国是世界上历史最悠久的国家之一。中国各族人民共同创造了光辉灿烂的文化，具有光荣的革命传统。

一八四○年以后，封建的中国逐渐变成半殖民地、半封建的国家。中国人民为国家独立、民族解放和民主自由进行了前仆后继的英勇奋斗。

二十世纪，中国发生了翻天覆地的伟大历史变革。

一九一一年孙中山先生领导的辛亥革命，废除了封建帝制，创立了中华民国。但是，中国人民反对帝国主义和封建主义的历史任务还没有完成。

一九四九年，以毛泽东主席为领袖的中国共产党领导中国各族人民，在经历了长期的艰难曲折的武装斗争和其他形式的斗争以后，终于推翻了帝国主义、封建主义和官僚资本主义的统治，取得了新民主主义革命的伟大胜利，建立了中华人民共和国。从此，中国人民掌握了国家的权力，成为国家的主人。

<!-- FORCE BREAK -->

中华人民共和国成立以后，我国社会逐步实现了由新民主主义到社会主义的过渡。生产资料私有制的社会主义改造已经完成，人剥削人的制度已经消灭，社会主义制度已经确立。工人阶级领导的、以工农联盟为基础的人民民主专政，实质上即无产阶级专政，得到巩固和发展。中国人民和中国人民解放军战胜了帝国主义、霸权主义的侵略、破坏和武装挑衅，维护了国家的独立和安全，增强了国防。经济建设取得了重大的成就，独

In [11]:
documents

[Document(id_='60025d69-bb9d-42e6-ad78-8c987906ff92', embedding=None, metadata={'head1': '中华人民共和国宪法'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='93ffd04f-7317-4f0c-aa8e-bfad11c1daa7', embedding=None, metadata={'head1': '中华人民共和国宪法', 'head2': '序言'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1982年12月4日 第五届全国人民代表大会第五次会议通过\n\n1982年12月4日 全国人民代表大会公告公布施行\n\n1988年4月12日 第七届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》\n\n1993年3月29日 第八届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》\n\n1999年3月15日 第九届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》\n\n2004年3月14日 第十届全国人民代表大会第二次会议通过的《中华人民共和国宪法修正案》\n\n2018年3月11日 第十三届全国人民代表大会第一次会议通过的《中华人民共和国宪法修正案》\n\n<!-- INFO END -->', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}',